In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import re
import urllib.request

In [3]:
# Cargar datos
df = pd.read_csv(r'C:\Users\alber\OneDrive\Documentos\GitHub\DS_ONLINE_THEBRIDGE_ATC\Masterclass\Kaggle\data\train.csv', index_col="laptop_ID")

In [4]:
df.head()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
laptop_ID,,,,,,,,,,,,
755,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.86kg,539.00
618,Dell,Inspiron 7559,Gaming,15.6,Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16GB,1TB HDD,Nvidia GeForce GTX 960<U+039C>,Windows 10,2.59kg,879.01
909,HP,ProBook 450,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,Nvidia GeForce 930MX,Windows 10,2.04kg,900.00
2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
286,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,AMD Radeon R5 M430,Linux,2.25kg,428.00


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 755 to 229
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           912 non-null    object 
 1   Product           912 non-null    object 
 2   TypeName          912 non-null    object 
 3   Inches            912 non-null    float64
 4   ScreenResolution  912 non-null    object 
 5   Cpu               912 non-null    object 
 6   Ram               912 non-null    object 
 7   Memory            912 non-null    object 
 8   Gpu               912 non-null    object 
 9   OpSys             912 non-null    object 
 10  Weight            912 non-null    object 
 11  Price_in_euros    912 non-null    float64
dtypes: float64(2), object(10)
memory usage: 92.6+ KB


In [6]:
df.describe()

,Inches,Price_in_euros
count,912.000000,912.000000
mean,14.981579,1111.724090
std,1.436719,687.959172
min,10.100000,174.000000
25%,14.000000,589.000000
50%,15.600000,978.000000
75%,15.600000,1483.942500
max,18.400000,6099.000000


In [7]:
df.tail()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
laptop_ID,,,,,,,,,,,,
28,Dell,Inspiron 5570,Notebook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,AMD Radeon 530,Windows 10,2.2kg,800.00
1160,HP,Spectre Pro,2 in 1 Convertible,13.3,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.48kg,1629.00
78,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,2TB HDD,Intel HD Graphics 620,No OS,2.2kg,519.00
23,HP,255 G6,Notebook,15.6,1366x768,AMD E-Series E2-9000e 1.5GHz,4GB,500GB HDD,AMD Radeon R2,No OS,1.86kg,258.00
229,Dell,Alienware 17,Gaming,17.3,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1060,Windows 10,4.42kg,2456.34


In [8]:
# Procesamiento de datos
df["Ram"] = df["Ram"].astype(str).str.replace("GB", "").astype(int)
df["Weight"] = df["Weight"].astype(str).str.replace("kg", "").astype(float)
df["Cpu_Brand"] = df["Cpu"].astype(str).apply(lambda x: x.split()[0])

# Seleccionar características y variable objetivo
X = df.drop(columns=["Price_in_euros", "Product", "Cpu"])
y = df["Price_in_euros"]

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identificar variables categóricas y numéricas
categorical_features = ["Company", "TypeName", "ScreenResolution", "Gpu", "OpSys", "Cpu_Brand", "Memory"]
numeric_features = ["Inches", "Ram", "Weight"]

# Preprocesador
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numeric_features),
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
])

# Modelo base con ajuste de hiperparámetros
param_grid = {
    "regressor__n_estimators": [100, 200, 300],
    "regressor__max_depth": [10, 15, 20],
    "regressor__min_samples_split": [2, 4, 6]
}

pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", RandomForestRegressor(random_state=42))
])

# Búsqueda de mejores hiperparámetros
grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring="neg_root_mean_squared_error", verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mejor modelo
best_model = grid_search.best_estimator_

# Evaluación
y_pred = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Mejor RMSE: {rmse}")




Fitting 3 folds for each of 27 candidates, totalling 81 fits
Mejor RMSE: 375.07330376137327


In [9]:
# Cargar conjunto de test y preprocesar
X_pred = pd.read_csv(r'C:\Users\alber\OneDrive\Documentos\GitHub\DS_ONLINE_THEBRIDGE_ATC\Masterclass\Kaggle\data\test.csv', index_col="laptop_ID")
X_pred["Ram"] = X_pred["Ram"].astype(str).str.replace("GB", "").astype(int)
X_pred["Weight"] = X_pred["Weight"].astype(str).str.replace("kg", "").astype(float)
X_pred["Cpu_Brand"] = X_pred["Cpu"].astype(str).apply(lambda x: x.split()[0])
X_pred = X_pred.drop(columns=["Product", "Cpu"])

# Predicciones en el conjunto de test
predictions = best_model.predict(X_pred)


In [10]:
predictions

array([1712.6988226 ,  304.13911138,  420.00632262, 1061.48777171,
        858.85081227,  509.43772843,  750.00430158,  993.6908052 ,
       1347.41740571,  384.04272466, 2280.78572045, 1513.86818158,
        586.76301792, 1609.43163014,  676.50739583,  548.20406757,
       2641.152935  , 1411.47530886, 1731.5125497 ,  572.41355363,
       1715.91562454,  405.7716805 ,  690.65111667, 1304.76450833,
        547.43529652,  734.32141509,  580.11564167,  630.59749982,
       2942.77445758, 1153.81473595, 2358.08084722,  455.92379208,
        674.2218    , 3216.2249    , 2023.36091061, 1509.7821704 ,
        546.35234251, 1518.46475536,  864.23397384, 1569.97671738,
        783.37537781,  918.3285    ,  554.39732024, 1251.61495353,
       1167.65813889, 1144.59142343, 1120.58498729,  678.19934516,
        715.45885942,  424.76101045, 1739.29128258,  734.56891121,
       1133.92797187,  621.45973429, 1710.98314136, 1717.15011985,
        656.49816915,  840.48966483, 1067.16242828,  629.47244

In [11]:
sample = pd.DataFrame({"laptop_ID": X_pred.index, "Price_in_euros": predictions})

In [12]:
sample.head()

,laptop_ID,Price_in_euros
0,209,1712.698823
1,1281,304.139111
2,1168,420.006323
3,1231,1061.487772
4,1020,858.850812


In [13]:
sample.shape

(391, 2)

In [14]:
submission = pd.DataFrame({"laptop_ID": X_pred.index, "Price_in_euros": predictions})

In [15]:
submission.head()

,laptop_ID,Price_in_euros
0,209,1712.698823
1,1281,304.139111
2,1168,420.006323
3,1231,1061.487772
4,1020,858.850812


In [16]:
submission.shape

(391, 2)

In [17]:
def chequeador(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                submission.to_csv("submission4.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://www.mihaileric.com/static/evaluation-meme-e0a350f278a36346e6d46b139b1d0da0-ed51e.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto del TA: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `test.csv`. Lloro.")

In [18]:
chequeador(submission)

You're ready to submit!


NameError: name 'Image' is not defined